In [2]:
#Ici , on va applquer les algorithmes de classification de machines learning pour déter²miner si un film est un succès commercial ou non
#On va d'abord transformé le dataset pour un apprentissage supervisé pour appliquer les algorithmes de regression
#Un film a succès commercial est un film qui a rapporté plus de la moyenne du revenu mois les budgets de production
import pandas as pd

df=pd.read_csv('movie_dataset_cleaned_final.csv')

#Afficher la moyenne du budget
print("Moyenne du budget:", df['budget'].mean())
#Moyenne des revenus
print("Moyenne des revenus:", df['revenue'].mean())
#Moyenne des revenues moins le budget 
print("Moyenne des revenus moins le budget:", (df['revenue'] - df['budget']).mean())
print("Revenues pour etre un film à succès commercial:", df['revenue'].mean() - df['budget'].mean())



Moyenne du budget: 29045039.87528628
Moyenne des revenus: 82260638.65167603
Moyenne des revenus moins le budget: 53215598.776389755
Revenues pour etre un film à succès commercial: 53215598.77638975


In [3]:
#Maintenant on transforme le dataset pour un apprentissage supervisé
#On ajoute une colonne "success" qui vaut 1 si le film est un succès commercial
df['success'] = ((df['revenue'] - df['budget']) > (df['revenue'].mean() - df['budget'].mean())).astype(int)
print(df[['budget', 'revenue', 'success']].head())

#On sauvegarde le nouveau dataset 
df.to_csv('movie_dataset_supervised.csv', index=False)




      budget     revenue  success
0  237000000  2787965087        1
1  300000000   961000000        1
2  245000000   880674609        1
3  250000000  1084939099        1
4  260000000   284139100        0


In [12]:
#Maintennant on va supprimer les colonnes inutiles pour l'apprentissage supervisé
columns_to_drop = ['genres', 'production_companies', 'production_countries', 'spoken_languages', 'cast']
df_supervised = df.drop(columns=columns_to_drop)
df_supervised.to_csv('movie_dataset_supervised_cleaned.csv', index=False)
#On sauvegarde le nouveau dataset nettoyé 

print("Nouveau dataset pour apprentissage supervisé sauvegardé sous 'movie_dataset_supervised_cleaned.csv'")

Nouveau dataset pour apprentissage supervisé sauvegardé sous 'movie_dataset_supervised_cleaned.csv'


In [13]:
#Affichage
import pandas
df1= pandas.read_csv('movie_dataset_supervised_cleaned.csv')

print(df1.head())

      budget original_language  popularity     revenue  runtime  vote_average  \
0  237000000                en  150.437577  2787965087    162.0           7.2   
1  300000000                en  139.082615   961000000    169.0           6.9   
2  245000000                en  107.376788   880674609    148.0           6.3   
3  250000000                en  112.312950  1084939099    165.0           7.6   
4  260000000                en   43.926995   284139100    132.0           6.1   

   vote_count           director  success  
0       11800      James Cameron        1  
1        4500     Gore Verbinski        1  
2        4466         Sam Mendes        1  
3        9106  Christopher Nolan        1  
4        2124     Andrew Stanton        0  


In [24]:
# -----------------------------
# Classification : Arbre de Décision
# -----------------------------

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# --- 1. Encodage des variables catégorielles ---
# Conversion des colonnes texte en variables numériques
df_supervised_encoded = pd.get_dummies(df_supervised, drop_first=True)

# --- 2. Séparation variables explicatives / variable cible ---
X = df_supervised_encoded.drop('success', axis=1)
y = df_supervised_encoded['success']

# --- 3. Split des données ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42
)

# --- 4. Création du modèle ---
model = DecisionTreeClassifier()

# --- 5. Entraînement ---
model.fit(X_train, y_train)

# --- 6. Prédiction ---
y_pred = model.predict(X_test)

# --- 7. Évaluation du modèle ---
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

#Nombre de correctements classifiés
correct_predictions = (y_test == y_pred).sum()
total_predictions = y_test.shape[0]
print(f"Nombre de prédictions correctes: {correct_predictions} sur {total_predictions}")

#Score de précision
accuracy = correct_predictions / total_predictions
print(f"Précision du modèle: {accuracy:.2f}")



Confusion Matrix:
[[1732   32]
 [  23  615]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98      1764
         1.0       0.95      0.96      0.96       638

    accuracy                           0.98      2402
   macro avg       0.97      0.97      0.97      2402
weighted avg       0.98      0.98      0.98      2402

Nombre de prédictions correctes: 2347 sur 2402
Précision du modèle: 0.98


In [25]:
#Maintenant on va appliquer random forest pour la classification
from sklearn.ensemble import RandomForestClassifier
# --- 4. Création du modèle ---
model = RandomForestClassifier(n_estimators=100, random_state=42)
# --- 5. Entraînement ---
model.fit(X_train, y_train)
# --- 6. Prédiction ---
y_pred = model.predict(X_test)
# --- 7. Évaluation du modèle ---
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

#Nombre de correctements classifiés
correct_predictions = (y_test == y_pred).sum()
total_predictions = y_test.shape[0]
print(f"Nombre de prédictions correctes: {correct_predictions} sur {total_predictions}")

#Score de précision
accuracy = correct_predictions / total_predictions
print(f"Précision du modèle: {accuracy:.2f}")

Confusion Matrix:
[[1732   32]
 [  44  594]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      1764
         1.0       0.95      0.93      0.94       638

    accuracy                           0.97      2402
   macro avg       0.96      0.96      0.96      2402
weighted avg       0.97      0.97      0.97      2402

Nombre de prédictions correctes: 2326 sur 2402
Précision du modèle: 0.97


In [23]:
# Maintenant on va appliquer le classifieur bayésien (Naive Bayes)
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import pandas as pd

# --- 1. Charger le dataset supervisé ---
df_supervised = pd.read_csv('movie_dataset_supervised_cleaned.csv')

# --- 2. Imputer les valeurs manquantes ---
imputer = SimpleImputer(strategy='mean')
for col in df_supervised.select_dtypes(include=['float64', 'int64']).columns:
    df_supervised[col] = imputer.fit_transform(df_supervised[[col]])

# --- 3. Encodage des variables catégorielles ---
df_supervised_encoded = pd.get_dummies(df_supervised, drop_first=True)

# --- 4. Séparation variables explicatives / variable cible ---
X = df_supervised_encoded.drop('success', axis=1)
y = df_supervised_encoded['success']

# --- 5. Split des données ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42
)

# --- 6. Création du modèle ---
model = GaussianNB()

# --- 7. Entraînement ---
model.fit(X_train, y_train)

# --- 8. Prédiction ---
y_pred = model.predict(X_test)

# --- 9. Évaluation du modèle ---
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Nombre de correctements classifiés
correct_predictions = (y_test == y_pred).sum()
total_predictions = y_test.shape[0]
print(f"Nombre de prédictions correctes: {correct_predictions} sur {total_predictions}")

# Score de précision
accuracy = correct_predictions / total_predictions
print(f"Précision du modèle: {accuracy:.2f}")

Confusion Matrix:
[[1692   72]
 [ 114  524]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.96      0.95      1764
         1.0       0.88      0.82      0.85       638

    accuracy                           0.92      2402
   macro avg       0.91      0.89      0.90      2402
weighted avg       0.92      0.92      0.92      2402

Nombre de prédictions correctes: 2216 sur 2402
Précision du modèle: 0.92


In [22]:
# Maintenant on va appliquer K-Nearest Neighbors (KNN)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import pandas as pd

# --- 1. Charger le dataset supervisé ---
df_supervised = pd.read_csv('movie_dataset_supervised_cleaned.csv')

# --- 2. Imputer les valeurs manquantes ---
imputer = SimpleImputer(strategy='mean')
for col in df_supervised.select_dtypes(include=['float64', 'int64']).columns:
    df_supervised[col] = imputer.fit_transform(df_supervised[[col]])

# --- 3. Encodage des variables catégorielles ---
df_supervised_encoded = pd.get_dummies(df_supervised, drop_first=True)

# --- 4. Séparation variables explicatives / variable cible ---
X = df_supervised_encoded.drop('success', axis=1)
y = df_supervised_encoded['success']

# --- 5. Split des données ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42
)

# --- 6. Création du modèle ---
model = KNeighborsClassifier(n_neighbors=5)

# --- 7. Entraînement ---
model.fit(X_train, y_train)

# --- 8. Prédiction ---
y_pred = model.predict(X_test)

# --- 9. Évaluation du modèle ---
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Nombre de correctements classifiés
correct_predictions = (y_test == y_pred).sum()
total_predictions = y_test.shape[0]
print(f"Nombre de prédictions correctes: {correct_predictions} sur {total_predictions}")

# Score de précision
accuracy = correct_predictions / total_predictions
print(f"Précision du modèle: {accuracy:.2f}")

Confusion Matrix:
[[1760    4]
 [   9  629]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      1764
         1.0       0.99      0.99      0.99       638

    accuracy                           0.99      2402
   macro avg       0.99      0.99      0.99      2402
weighted avg       0.99      0.99      0.99      2402

Nombre de prédictions correctes: 2389 sur 2402
Précision du modèle: 0.99
